<a href="https://colab.research.google.com/github/ouriquco/Classificiation-Academic-Success/blob/summer_models/Academic_Success.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [45]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict

# Reading in Data

In [46]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

# Data Preprocessing

In [47]:
#checking for NA
train.isnull().sum()
test.isnull().sum()

#checking for duplicates
train.duplicated().sum()
test.duplicated().sum()

features = test.columns

#convert categorical data to numeric
if train['Target'].dtype == 'object':
    le = LabelEncoder()
    train['Target'] = le.fit_transform(train['Target'])

#splitting into features and target
X_train = train[features]
y_train = train['Target']
X_test = test[features]

# Data Visualization

# Cross Validation

# Random Forest

In [ ]:

#best parameters
print("Best Parameters:", grid_search.best_params_)
#initialize Random Forest - play with inputs here
#give more weight to the second class
rf = RandomForestClassifier()

# Fit the model on the training data
rf.fit(X_train, y_train)

# Feature importance
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': rf.feature_importances_})
feature_importance.sort_values(by='Importance', ascending=False, inplace=True)

#filtering out less important features
features_mask = rf.feature_importances_ > 0.01
X_train_reduced = X_train.loc[:, features_mask]
X_test_reduced = X_test.loc[:, features_mask]

#parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_train_reduced, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Fit the model on the reduced training data
best_model.fit(X_train_reduced, y_train)

# Predict on the test data
y_pred = best_model.predict(X_test_reduced)

#cross validation
cv_scores = cross_val_score(best_model, X_train_reduced, y_train, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Average Cross-Validation Score:", cv_scores.mean())

#cross validation confusion matrix
train_pred = cross_val_predict(best_model, X_train_reduced, y_train, cv=5)
cm = confusion_matrix(y_train, train_pred)
print("Confusion Matrix:")
print(cm)

#create submission file
#convert target back from numbers into "graduate", "dropout", or "enrolled"
labels = le.inverse_transform(y_pred)

submission = pd.DataFrame({'id': test.index, 'Target': labels})
submission.to_csv('submission.csv', index=False)


# AdaBoost

# Gradient Boosting

# XGBoost

## CatBoost

# LightGBM